<a href="https://colab.research.google.com/github/mdsadaqathali5/lab-records/blob/main/problem7(54).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###1)The probability that it is friday and that a student is absent 3%.Since there are five school days in a week,the probability that it is friday is 20%.What is the probability that a student is absent given that today is friday?Apply Baye's rule in python to get the result.


In [1]:
probAbsentFriday = 0.03
probFriday = 0.2

# Bayes Formula
# p(Absent|Friday)=p(Friday|Absent)p(Absent)/p(Friday)

bayesResult = (probAbsentFriday/probFriday)
print(bayesResult*100)

15.0


###2)Write a program to implement the Naive Bayesian classifier for a sample training dataset stored as a .CSV file.Compute the accuracy of the classifier,considering few test datasets.


In [2]:
# Gaussian Naive Bayes
from sklearn import datasets
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
#load the iris datasets
dataset = datasets.load_iris()
#fit a Naive Bayes model to the data
model = GaussianNB()

model.fit(dataset.data,dataset.target)
print(model)

#make predictions
expected = dataset.target
predicted = model.predict(dataset.data)
#summarize the fit of the model
print(metrics.classification_report(expected,predicted))
print(metrics.confusion_matrix(expected,predicted))


GaussianNB()
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.94      0.94      0.94        50
           2       0.94      0.94      0.94        50

    accuracy                           0.96       150
   macro avg       0.96      0.96      0.96       150
weighted avg       0.96      0.96      0.96       150

[[50  0  0]
 [ 0 47  3]
 [ 0  3 47]]
